# Configuration Files and Hardware Portability

This notebook covers `nidaqwrapper`'s TOML-based configuration system, which allows you to:

1. **Serialize task configurations** to human-readable TOML files
2. **Recreate tasks** from saved configurations
3. **Move configurations between machines** by editing a single `[devices]` section

This is essential for deployment scenarios where device names change between development and production systems.

## Setup and Mock Detection

This cell attempts to import `nidaqmx`. If unavailable, it sets up mocks so the notebook runs on any machine.

In [ ]:
import sys
import tempfile
from pathlib import Path
from unittest.mock import MagicMock, PropertyMock

# Try to import nidaqmx
try:
    import nidaqmx
    from nidaqmx import constants
    HW_AVAILABLE = True
    print("\u2705 Real NI-DAQmx hardware available")
except ImportError:
    print("\u26a0\ufe0f NI-DAQmx not found - using mocks")
    HW_AVAILABLE = False
    
    # Create mock nidaqmx module structure
    nidaqmx = MagicMock()
    sys.modules['nidaqmx'] = nidaqmx
    sys.modules['nidaqmx.system'] = nidaqmx.system
    sys.modules['nidaqmx.task'] = nidaqmx.task
    sys.modules['nidaqmx.constants'] = nidaqmx.constants
    
    # Mock constants
    constants = nidaqmx.constants
    constants.AcquisitionType = MagicMock()
    constants.AcquisitionType.CONTINUOUS = MagicMock()
    constants.TerminalConfiguration = MagicMock()
    constants.TerminalConfiguration.DEFAULT = MagicMock()
    constants.VoltageUnits = MagicMock()
    constants.VoltageUnits.VOLTS = MagicMock()
    constants.AccelUnits = MagicMock()
    constants.AccelUnits.G = MagicMock()
    constants.AccelSensitivityUnits = MagicMock()
    constants.AccelSensitivityUnits.M_VOLTS_PER_G = MagicMock()
    constants.LineGrouping = MagicMock()
    constants.LineGrouping.CHAN_PER_LINE = MagicMock()
    constants.READ_ALL_AVAILABLE = -1
    constants.RegenerationMode = MagicMock()
    constants.RegenerationMode.ALLOW_REGENERATION = MagicMock()
    
    # Mock device objects
    mock_dev1 = MagicMock()
    mock_dev1.name = "Dev1"
    mock_dev1.product_type = "PCIe-6320"
    
    mock_dev2 = MagicMock()
    mock_dev2.name = "cDAQ1Mod1"
    mock_dev2.product_type = "NI 9234"
    
    mock_di_line = MagicMock()
    mock_di_line.name = "Dev1/port0/line0"
    mock_dev1.di_lines = [mock_di_line]
    mock_dev1.do_lines = [mock_di_line]
    
    # Mock system
    mock_system = MagicMock()
    mock_system.devices = [mock_dev1, mock_dev2]
    mock_system.tasks.task_names = []
    nidaqmx.system.System.local.return_value = mock_system
    
    # Mock Task
    def create_mock_task(new_task_name=None):
        mock_task = MagicMock()
        mock_task.name = new_task_name or "mock_task"
        mock_task.channel_names = []
        mock_task.ai_channels = []
        mock_task.ao_channels = []
        mock_task.di_channels = []
        mock_task.do_channels = []
        
        # Mock add_ai_accel_chan
        def add_ai_accel_chan(**kwargs):
            mock_ch = MagicMock()
            mock_ch.name = kwargs.get('name_to_assign_to_channel', 'channel')
            mock_ch.physical_channel.name = kwargs.get('physical_channel', '')
            mock_task.channel_names.append(mock_ch.name)
            mock_task.ai_channels.append(mock_ch)
        
        # Mock add_ao_voltage_chan
        def add_ao_voltage_chan(**kwargs):
            mock_ch = MagicMock()
            mock_ch.name = kwargs.get('name_to_assign_to_channel', 'channel')
            mock_ch.physical_channel.name = kwargs.get('physical_channel', '')
            mock_task.channel_names.append(mock_ch.name)
            mock_task.ao_channels.append(mock_ch)
        
        # Mock add_di_chan
        def add_di_chan(**kwargs):
            mock_ch = MagicMock()
            mock_ch.name = kwargs.get('name_to_assign_to_lines', 'channel')
            mock_ch.physical_channel.name = kwargs.get('lines', '')
            mock_task.channel_names.append(mock_ch.name)
            mock_task.di_channels.append(mock_ch)
        
        # Mock add_do_chan
        def add_do_chan(**kwargs):
            mock_ch = MagicMock()
            mock_ch.name = kwargs.get('name_to_assign_to_lines', 'channel')
            mock_ch.physical_channel.name = kwargs.get('lines', '')
            mock_task.channel_names.append(mock_ch.name)
            mock_task.do_channels.append(mock_ch)
        
        mock_task.ai_channels.add_ai_accel_chan = add_ai_accel_chan
        mock_task.ai_channels.add_ai_voltage_chan = add_ai_accel_chan
        mock_task.ao_channels.add_ao_voltage_chan = add_ao_voltage_chan
        mock_task.di_channels.add_di_chan = add_di_chan
        mock_task.do_channels.add_do_chan = add_do_chan
        
        # Mock timing
        mock_task.timing.cfg_samp_clk_timing = MagicMock()
        mock_task.timing.samp_clk_rate = 1000.0
        
        return mock_task
    
    nidaqmx.task.Task.side_effect = create_mock_task
    nidaqmx.Task = nidaqmx.task.Task

# Create a temporary directory for config files
temp_dir = tempfile.mkdtemp()
config_dir = Path(temp_dir)
print(f"Config files will be saved to: {config_dir}")

## Import nidaqwrapper

In [ ]:
from nidaqwrapper import AITask, AOTask, DITask, DOTask

## Saving Analog Input Configuration

The `save_config()` method serializes a task's configuration to a TOML file. All channel parameters, device mappings, and timing settings are preserved.

In [ ]:
# Create an analog input task with two accelerometer channels
ai_task = AITask("vibration_test", sample_rate=25600)
ai_task.add_channel(
    "accel_x",
    device_ind=0,
    channel_ind=0,
    sensitivity=100.0,
    sensitivity_units="mV/g",
    units="g",
    min_val=-5.0,
    max_val=5.0,
)
ai_task.add_channel(
    "accel_y",
    device_ind=1,
    channel_ind=2,
    sensitivity=102.5,
    sensitivity_units="mV/g",
    units="g",
    min_val=-10.0,
    max_val=10.0,
)

# Save configuration to TOML
ai_config_path = config_dir / "vibration_test.toml"
ai_task.save_config(ai_config_path)
ai_task.clear_task()

print(f"Configuration saved to: {ai_config_path}")

## Understanding the TOML Structure

Let's examine the generated TOML file:

In [ ]:
print(ai_config_path.read_text())

### TOML Structure Explained

The configuration file has three main sections:

#### 1. Header Comment
```toml
# Generated by nidaqwrapper 0.1.0 on 2026-02-19 10:30
```
- Identifies the generator and timestamp
- Human-readable metadata

#### 2. `[task]` Section
```toml
[task]
name = "vibration_test"
sample_rate = 25600
type = "input"
```
- **name**: Task name (must be unique in NI MAX)
- **sample_rate**: Sampling rate in Hz
- **type**: One of `"input"`, `"output"`, `"digital_input"`, `"digital_output"`

#### 3. `[devices]` Section (Hardware Portability)
```toml
[devices]
dev0 = "Dev1"  # PCIe-6320
dev1 = "cDAQ1Mod1"  # NI 9234
```
- Maps **device aliases** (dev0, dev1) to actual device names
- Includes product type as a comment for reference
- **This is the only section you need to edit when moving between machines**

#### 4. `[[channels]]` Array
```toml
[[channels]]
name = "accel_x"
device = "dev0"  # References the alias, not the device name
channel = 0
sensitivity = 100.0
sensitivity_units = "mV/g"
units = "g"
min_val = -5.0
max_val = 5.0
```
- One `[[channels]]` entry per channel
- Uses device **aliases** instead of indices
- All original `add_channel()` parameters preserved

## Loading Configuration with `from_config()`

The `from_config()` classmethod recreates a task from a saved TOML file. It:
1. Parses the TOML file
2. Resolves device aliases to current system device indices
3. Creates the task and adds all channels
4. Returns a fully configured task (not yet started)

In [ ]:
# Recreate the task from the config file
ai_task_restored = AITask.from_config(ai_config_path)

print(f"Restored task: {ai_task_restored.task_name}")
print(f"Sample rate: {ai_task_restored.sample_rate} Hz")
print(f"Channels: {ai_task_restored.channel_list}")

# The task is ready to start
# ai_task_restored.start(start_task=True)
# data = ai_task_restored.acquire_base()

ai_task_restored.clear_task()

## Hardware Portability: Device Alias Mapping

### The Problem
Device names change between machines:
- Development PC: `"Dev1"`, `"Dev2"`
- Test bench: `"cDAQ1Mod1"`, `"cDAQ1Mod2"`
- Production system: `"PXI1Slot2"`, `"PXI1Slot3"`

### The Solution
Device **aliases** decouple channel configurations from physical device names:
```toml
[devices]
dev0 = "Dev1"  # On development PC
dev1 = "cDAQ1Mod1"

[[channels]]
device = "dev0"  # References the alias, not "Dev1" directly
```

When you move to a new machine:
1. Open the TOML file in a text editor
2. Update **only** the `[devices]` section
3. No code changes needed

### Example: Moving to Production
```toml
[devices]
dev0 = "PXI1Slot2"  # Updated device name
dev1 = "PXI1Slot3"  # Updated device name

[[channels]]
device = "dev0"  # Same alias, different hardware
```

## Error Handling: Missing Device

If a device from the config is not present on the current system, `from_config()` raises `RuntimeError` (FR-2.11):

In [ ]:
# Create a config with a non-existent device (unless in mock mode)
if HW_AVAILABLE:
    # This will only work with real hardware
    bad_config = config_dir / "missing_device.toml"
    bad_config.write_text("""
# Test config with missing device

[task]
name = "test_task"
sample_rate = 1000
type = "input"

[devices]
dev0 = "NonExistentDevice99"

[[channels]]
name = "ch0"
device = "dev0"
channel = 0
units = "V"
""")
    
    try:
        AITask.from_config(bad_config)
    except RuntimeError as e:
        print(f"Expected error: {e}")
else:
    print("Skipped (mock mode)")

## Analog Output Configuration

AOTask configs include the `samples_per_channel` buffer size:

In [ ]:
# Create an analog output task
ao_task = AOTask("signal_gen", sample_rate=10000, samples_per_channel=50000)
ao_task.add_channel("ao_0", device_ind=0, channel_ind=0, min_val=-5.0, max_val=5.0)
ao_task.add_channel("ao_1", device_ind=0, channel_ind=1, min_val=-10.0, max_val=10.0)

# Save and display
ao_config_path = config_dir / "signal_gen.toml"
ao_task.save_config(ao_config_path)
ao_task.clear_task()

print("Analog Output Configuration:")
print("=" * 60)
print(ao_config_path.read_text())

### Key Differences in AOTask TOML
- **type**: `"output"` instead of `"input"`
- **samples_per_channel**: Buffer size (not present in AITask)
- **min_val/max_val**: Always written (have defaults of -10.0 and 10.0)

In [ ]:
# Restore AOTask
ao_task_restored = AOTask.from_config(ao_config_path)
print(f"Restored: {ao_task_restored.task_name}")
print(f"Buffer: {ao_task_restored.samples_per_channel} samples/channel")
ao_task_restored.clear_task()

## Digital I/O Configuration

Digital tasks use **line specifications** instead of device indices, so they don't have a `[devices]` section:

In [ ]:
# Digital Input (on-demand mode)
di_task = DITask("switches")  # No sample_rate = on-demand
di_task.add_channel("sw_0_3", lines="Dev1/port0/line0:3")

di_config_path = config_dir / "switches.toml"
di_task.save_config(di_config_path)
di_task.clear_task()

print("Digital Input Configuration (on-demand):")
print("=" * 60)
print(di_config_path.read_text())

### Key Differences in Digital TOML
- **type**: `"digital_input"` or `"digital_output"`
- **No `[devices]` section**: Line specs are absolute (e.g., `"Dev1/port0/line0:3"`)
- **sample_rate**: Omitted for on-demand mode, present for clocked mode
- **lines**: Full line specification string

In [ ]:
# Digital Output (clocked mode)
do_task = DOTask("pattern_gen", sample_rate=1000)  # Clocked mode
do_task.add_channel("leds", lines="Dev1/port1/line0:7")

do_config_path = config_dir / "pattern_gen.toml"
do_task.save_config(do_config_path)
do_task.clear_task()

print("Digital Output Configuration (clocked):")
print("=" * 60)
print(do_config_path.read_text())

In [ ]:
# Restore digital tasks
di_task_restored = DITask.from_config(di_config_path)
print(f"DI restored: {di_task_restored.task_name} (mode: {di_task_restored.mode})")
di_task_restored.clear_task()

do_task_restored = DOTask.from_config(do_config_path)
print(f"DO restored: {do_task_restored.task_name} (mode: {do_task_restored.mode})")
do_task_restored.clear_task()

## Editing TOML Files Manually

TOML is designed to be human-readable and editable. Common edits:

### 1. Change device mapping (portability)
```toml
[devices]
dev0 = "cDAQ2Mod1"  # Changed from "Dev1"
```

### 2. Adjust channel parameters
```toml
[[channels]]
name = "accel_x"
sensitivity = 105.0  # Updated calibration value
```

### 3. Change sample rate
```toml
[task]
sample_rate = 51200  # Increased from 25600
```

### 4. Rename task
```toml
[task]
name = "vibration_production"  # Renamed for deployment
```

## Practical Workflow Example

A typical deployment workflow:

1. **Development**: Create and test task
   ```python
   task = AITask("sensor_array", sample_rate=25600)
   task.add_channel("sensor_1", device_ind=0, channel_ind=0, ...)
   task.save_config("sensor_array.toml")
   ```

2. **Pre-deployment**: Identify production device names
   ```python
   from nidaqwrapper.utils import list_devices
   devices = list_devices()
   # devices = ['PXI1Slot2', 'PXI1Slot3']
   ```

3. **Deployment**: Edit TOML `[devices]` section
   ```toml
   [devices]
   dev0 = "PXI1Slot2"  # Updated from "Dev1"
   dev1 = "PXI1Slot3"  # Updated from "cDAQ1Mod1"
   ```

4. **Production**: Load and run
   ```python
   task = AITask.from_config("sensor_array.toml")
   task.start(start_task=True)
   ```

## Summary

TOML configuration provides:

1. **Serialization**: `task.save_config(path)` writes complete task configuration
2. **Deserialization**: `Task.from_config(path)` recreates tasks
3. **Hardware portability**: Device aliases decouple configs from physical devices
4. **Human-editable**: TOML is readable and editable with any text editor
5. **Version control friendly**: Plain text format works with git

### Task Type Differences
- **AITask**: `type = "input"`, has `[devices]` section
- **AOTask**: `type = "output"`, includes `samples_per_channel`
- **DITask**: `type = "digital_input"`, uses `lines`, no `[devices]`
- **DOTask**: `type = "digital_output"`, uses `lines`, no `[devices]`

### Error Handling
- Missing device: `RuntimeError` with device list (FR-2.11)
- Invalid TOML: `tomllib.TOMLDecodeError` propagated
- Missing sections: `ValueError` with clear message

## Cleanup

In [ ]:
import shutil
shutil.rmtree(temp_dir, ignore_errors=True)
print(f"Cleaned up temporary directory: {temp_dir}")